# Yash Deshpande C003 B1

### Installing Necessary Dependencies 

In [ ]:
!pip install mediapipe opencv-python

### Importing Modules needed

In [ ]:
import cv2,time,random
import mediapipe as mp

'''
cv2 For video input and processing
mediapipe is a ready to use ml solution by google
Random For randomly making a choice for the computer
Time for keeping track of time
'''


### Initializing Variables

In [ ]:

# First step is to initialize the Hands class an store it in a variable
mp_hands = mp.solutions.hands

# To set up the drawing function of hands landmarks on the image
mp_drawing = mp.solutions.drawing_utils


In [ ]:
'''Now to set the hands function which will hold the landmarks points
static_image_mode: is basicaly whether you want an image input or a video input
min_detection_confidence: This will detect hands with around 70% accuracy
'''
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1, min_detection_confidence=0.3)


### The Hand Landmark Model

In [ ]:
from IPython.display import Image
Image("hand_landmarks.png")

In [ ]:
'''
1-Rock
2-Paper
3-Scissors
'''
user=0
text=""
startofgame=False # To Start The Game
startoftime=0 #Making the Initial Timing global 
timing=0 #for storing time for the user to make his move 
score=[0,0] # Computer and Human
ai=False

### The Main Loop 

In [ ]:
cap = cv2.VideoCapture(0)  # Get the first posible webcam
cap.set(3, 720) # Changing width of video input
cap.set(4, 480) # Changing height of video input


while cap.isOpened(): # Running The Loop when video is loaded properly
    bg=cv2.imread("Rock Paper Scissors.png") # Opencv reading the background 
    
    cv2.putText(bg,"Enter s to play", (300, 1000), cv2.FONT_HERSHEY_SIMPLEX,2, (235, 111, 146),4)            
    cv2.putText(bg,"Enter q to stop", (1200, 1000), cv2.FONT_HERSHEY_SIMPLEX,2, (235, 111, 146),4)            
    cv2.putText(bg,f"{score[0],score[1]}", (900, 870) , cv2.FONT_HERSHEY_SIMPLEX,2, (235, 111, 146),4)            
    if ai: # This is for displaying images of rps when its a draw or when ai wins
        comp=cv2.imread(f"{randomnum}.png")
        width,height,k= comp.shape # Getting the tuple of rows, columns, and channels and unpacking them
        bg[500:width+500,300:height+300]=comp # Superimposing the respective image in the background by replacing the background matrix with that of that image of rps

    ret,frame=cap.read()

    '''
    Reading the video input.
    ret is  a boolean variable that returns true if the frame is available
    frame is an image array vector captured based on the default frames per second defined explicitly or implicitly
    '''

    # if frame is read correctly ret is True# First step is to initialize the Hands class an store it in a variable
    if not ret:
        print("Can't receive frame, terminating now ...")
        break
   
    frame = cv2.flip(frame, 1)  # For mirroring the video

    # opecv reads and interprets images in brg format (due to historic reasons) but mediapipe needs rgb so we convert it
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    if results.multi_hand_landmarks: # This a list of x,y,z coordinates of each landmark of the hand in the video input
        hand_landmarks = results.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                        ) # Drawing Landmarks on Hand
        for point in mp_hands.HandLandmark: # This is for getting y coordinates of TIPs and PIPs of each finger 
            indextip=hand_landmarks.landmark[point.INDEX_FINGER_TIP].y 
            middletip=hand_landmarks.landmark[point.MIDDLE_FINGER_TIP].y 
            ringtip=hand_landmarks.landmark[point.RING_FINGER_TIP].y 
            pinkytip=hand_landmarks.landmark[point.PINKY_TIP].y 

            indexpip=hand_landmarks.landmark[point.INDEX_FINGER_PIP].y 
            middlepip=hand_landmarks.landmark[point.MIDDLE_FINGER_PIP].y 
            ringpip=hand_landmarks.landmark[point.RING_FINGER_PIP].y 
            pinkypip=hand_landmarks.landmark[point.PINKY_PIP].y 

            #When index and ring fingers tips are above the pips it is scissors
            if middletip<middlepip and indextip<indexpip and ringtip>ringpip and pinkytip>pinkypip:
                user=3 #Scissors
            # When all the tips are bellow the pips
            elif middletip>middlepip and indextip>indexpip and ringtip>ringpip and pinkytip>pinkypip:
                user=1 # Rock
            # When all the tips are above the pips
            elif middletip<middlepip and indextip<indexpip and ringtip<ringpip and pinkytip<pinkypip:
                user=2 # Paper
            else:
                user=0
    
    if startofgame: # Starting the game only when user says so
        cv2.putText(bg,str(round((time.time() - startoftime))), (900, 700), cv2.FONT_HERSHEY_SIMPLEX,3, (235, 111, 146),4) #Displaying The Score         
        randomnum=random.randint(1,3) # Computers Choice
        ai=False # To remove the rps image for a new fresh game
        text="" # Removing the text on display after each game

        if (time.time() - startoftime) >= 3 :
            if (user == 1 and randomnum == 3) or (user == 2 and randomnum == 1) or (user == 3 and randomnum == 2):
                score[1]+=1
                text="You Win!"
                ai=True
            elif (user == 3 and randomnum == 1) or (user == 1 and randomnum == 2) or (user == 2 and randomnum == 3):
                score[0] += 1
                text="PC Wins!"
                ai=True
            elif (user == 1 and randomnum == 1) or (user == 2 and randomnum == 2) or (user == 3 and randomnum == 3):
                text="Draw!"
                ai=True
            startofgame=False

    cv2.putText(bg,text, (800, 600), cv2.FONT_HERSHEY_SIMPLEX,3, (235, 111, 146),4) # Puts anything in the text variable on display           
    cv2.imshow("Background",bg) # Displaying the background
    cv2.imshow('Player', frame)  # Displaying the video

    key=cv2.waitKey(10) # Waiting for 0.01 seconds for user to start the game or exit the game
    if key == ord('s'): # Starting the game
        startoftime=time.time() # Getting current secconds from the unix epoch January 1, 1970 in seconds
        startofgame=True
    if key == ord ('r'):
        score=[0,0   ]
    
    if key == ord('q'):  # quitting when user enters q to quit the program
        break
        
else:
    print("Video Not Opened,Exiting")


In [ ]:
cap.release()
cv2.destroyAllWindows()